<center>
<img src="http://www.bigdive.eu/wp-content/uploads/2012/05/logoBIGDIVE-01.png">
</center>

---

# PANDAS
(The initial version of this notebook was prepared by André Panisson for Big Dive 2016.)

---

In [ ]:
%pylab inline

In [ ]:
import pandas as pd
plt.rc('figure', figsize=(10, 6))

Series
======

Series is a one-dimensional labeled array capable of holding any data type (integers, strings, floating point numbers, Python objects, etc.). The axis labels are collectively referred to as the index.

In [ ]:
labels = ['a', 'b', 'c', 'd', 'e']
s = pd.Series(randn(5), index=labels)
s

In [ ]:
s.index

Operators that work in dictionaries also work on Series objects

In [ ]:
'b' in s

In [ ]:
s['b']

We can transform a Series in a dictionary, and a dictionary in a Series

In [ ]:
mapping = s.to_dict()
mapping

In [ ]:
s = pd.Series(mapping)
s

Slicing works in a Series the same way it works in numpy arrays

In [ ]:
s[:3]

## DataFrame

A Pandas DataFrame is a 2D collection of Series. Each Series of a DataFrame is a column.

This is generally the most commonly used pandas object.

In [ ]:
df = pd.DataFrame({'a': np.random.randn(6),
                'b': ['foo', 'bar'] * 3,
                'c': np.random.randn(6)})
df

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df = pd.DataFrame({'a': np.random.randn(6),
                'b': ['foo', 'bar'] * 3,
                'c': np.random.randn(6)},
               index=pd.date_range('1/1/2000', periods=6))
df

In [ ]:
df = pd.DataFrame({'a': np.random.randn(6),
                'b': ['foo', 'bar'] * 3,
                'c': np.random.randn(6)},
               columns=['a', 'b', 'c', 'd'])
df

Creation from nested dicts
--------------------------

These arise naturally in Python code

In [ ]:
data = {}
for col in ['foo', 'bar', 'baz']:
    for row in ['a', 'b', 'c', 'd']:
        data.setdefault(col, {})[row] = randn()
data

In [ ]:
pd.DataFrame(data)

Data alignment
==============

In [ ]:
close_px = pd.read_csv('stock_data.csv', index_col=0, parse_dates=True)

In [ ]:
close_px.head()

In [ ]:
s1 = close_px['AAPL'][-20:]
s2 = close_px['AAPL'][-25:-10]

def side_by_side(*objs, **kwds):
    #from pandas.core.common import adjoin
    from pandas.formats.printing import adjoin
    space = kwds.get('space', 4)
    reprs = [repr(obj).split('\n') for obj in objs]
    print adjoin(space, *reprs)
    
side_by_side(s1, s2)

In [ ]:
s1 + s2

In [ ]:
df = close_px.ix[-10:, :3]
df

In [ ]:
side_by_side(s1.reindex(s2.index), s2)

In [ ]:
b, c  = s1.align(s2, join='inner')
side_by_side(b, c)

In [ ]:
b, c  = s1.align(s2, join='outer')
side_by_side(b, c)

In [ ]:
b, c  = s1.align(s2, join='right')
side_by_side(b, c)

In [ ]:
b, c  = s1.align(s2, join='left')
side_by_side(b, c)

In [ ]:
df = close_px.ix[-10:, ['AAPL', 'IBM', 'MSFT']]
df

In [ ]:
df2 = df.ix[::2, ['IBM', 'MSFT']]
side_by_side(df, df2)

In [ ]:
df + df2

In [ ]:
b, c = df.align(df2, join='inner')
side_by_side(b, c) 

Transposing
-----------

In [ ]:
df[:5].T

Columns can be any type
-----------------------

In [ ]:
n = 10
foo = pd.DataFrame(index=range(n))
foo['floats'] = np.random.randn(n)
foo['ints'] = np.arange(n)
foo['strings'] = ['foo', 'bar'] * (n / 2)
foo['bools'] = foo['floats'] > 0
foo['objects'] = pd.date_range('1/1/2000', periods=n)
foo

In [ ]:
foo.dtypes

Function application
====================

You can apply arbitrary functions to the rows or columns of a DataFrame

In [ ]:
df.apply(np.mean)

In [ ]:
df.apply(np.mean, axis=1)

You can get as fancy as you want

In [ ]:
close_px.head()

In [ ]:
def peak_date(series):
    return series.index[series.values.argmax()]
close_px.apply(peak_date)

In [ ]:
df.apply(lambda x: x.max() - x.min()) 

In [ ]:
np.log(close_px)

Plotting
========

Some basic plotting integration with matplotlib in Series / DataFrame

In [ ]:
close_px[['AAPL', 'IBM', 'MSFT', 'XOM']].plot()

In [ ]:
close_px.ix[-1]

In [ ]:
close_px.ix[-1].plot(kind='bar')
title('Prices on %s' % close_px.index[-1])
axhline(0)

GroupBy
=======

In [ ]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                       'foo', 'bar', 'foo', 'foo'],
                'B' : ['one', 'one', 'two', 'three',
                       'two', 'two', 'one', 'three'],
                'C' : np.random.randn(8),
                'D' : np.random.randn(8)})
df

In [ ]:
for key, group in df.groupby('A'):
    print key
    print group

In [ ]:
df.groupby('A')['C'].describe().T

In [ ]:
df.groupby('A').mean()

In [ ]:
for key, group in df.groupby('A'):
    print key
    print group

In [ ]:
df.groupby(['A', 'B']).mean()

In [ ]:
df.groupby(['A', 'B'], as_index=False).mean()

---

## Analysis of the baby names dataset

In [ ]:
names = pd.read_csv('baby-names.csv')

In [ ]:
names.head()

In [ ]:
names.dtypes

In [ ]:
names[names.year == 1880]

In [ ]:
names[names.year == 1880].tail()

In [ ]:
boys = names[names.sex == 'boy']    
girls = names[names.sex == 'girl']

In [ ]:
boys.groupby('year')

In [ ]:
boys.groupby('year').size()

In [ ]:
names.groupby(['year', 'sex']).size()

In [ ]:
type(names.groupby(['year', 'sex']).size())

In [ ]:
names.groupby(['year', 'sex']).size().ix[2000]    

In [ ]:
boys[boys.year == 2000]   

In [ ]:
boys[boys.year == 2000].prop

In [ ]:
boys[boys.year == 2000][:5]

In [ ]:
boys[boys.year == 2000].prop.idxmax()

In [ ]:
boys.ix[boys[boys.year == 2000].prop.idxmax()] 

In [ ]:
def get_max_record(group):
    return group.ix[group.prop.idxmax()]

get_max_record(boys)

In [ ]:
def get_max_record(group):
    return group.ix[group.prop.idxmax()]

result = boys.groupby('year').apply(get_max_record)

In [ ]:
result

In [ ]:
result.prop.plot()

In [ ]:
boys[boys.name == 'Travis']

In [ ]:
idf = boys.set_index(['name', 'year'])

In [ ]:
idf[-50:]

In [ ]:
idf.ix['Travis']

In [ ]:
idf.ix['Travis'].prop.plot()

In [ ]:
boys.groupby('name')['prop'].mean()

In [ ]:
boys.groupby('name')['prop'].mean().sort_values()

In [ ]:
boys['prop'].describe() 

In [ ]:
result = boys.groupby('year')['prop'].describe()

In [ ]:
result[:50]

In [ ]:
df = boys[boys.year == 2008]

In [ ]:
df.prop

In [ ]:
df = boys[boys.year == 2008].sort_values(by='prop', ascending=False) 

In [ ]:
df.prop

In [ ]:
df.prop.cumsum() 

In [ ]:
df.prop.values.cumsum().searchsorted(0.5)   # Also used as a measure of diversity.

In [ ]:
df.prop.cumsum()[:130]

In [ ]:
def get_quantile_count(group, quantile = 0.5):
    df = group.sort_values(by='prop', ascending=False)
    return df.prop.values.cumsum().searchsorted(quantile)

boys.groupby('year').apply(get_quantile_count).plot()

In [ ]:
def get_quantile_count(group, quantile=0.5):
    group = group.groupby('soundex').sum()
    df = group.sort_values(by='prop', ascending=False)
    return df.prop.values.cumsum().searchsorted(quantile)

q = 0.25
boy_ct = boys.groupby('year').apply(get_quantile_count, quantile=q)   # pass different values for quantile
girl_ct = girls.groupby('year').apply(get_quantile_count, quantile=q)
boy_ct.plot(label='boy')
girl_ct.plot(label='girl')
legend(loc='best')

In [ ]:
boys[boys.year == 2008].prop.rank() 

In [ ]:
grouped = boys.groupby('year')['prop']

In [ ]:
grouped.transform(pd.Series.rank)

In [ ]:
boys['year_rank'] = grouped.transform(pd.Series.rank)

In [ ]:
boys[boys.name == 'Andrew'].year_rank.plot()

In [ ]:
names   

In [ ]:
births = pd.read_csv('births.csv')

In [ ]:
merged = pd.merge(names, births, on=['year', 'sex'])  

In [ ]:
pd.merge(names, births) 

In [ ]:
merged['persons'] = np.floor(merged.prop * merged.births)

In [ ]:
merged.head()

In [ ]:
merged.groupby(['name', 'sex'])['persons'].sum() 

In [ ]:
merged.groupby(['name', 'sex'])['persons'].sum().sort_values()

In [ ]:
mboys = pd.merge(boys, births) 

In [ ]:
mboys['persons'] = np.floor(mboys.prop * mboys.births)

In [ ]:
persons = mboys.set_index(['year', 'name']).persons 

In [ ]:
type(persons)  

In [ ]:
persons 

In [ ]:
persons.ix[:, 'Christopher'].plot(kind='bar', rot=90)  # Plot is crowded. Matplotlib doesn't go more than 130 in x axis.

---

Reproduction of [Quantifying Trading Behavior in Financial Markets Using Google Trends](http://www.nature.com/srep/2013/130425/srep01684/pdf/srep01684.pdf)
==============================================

In this section we will reproduce a paper published recently on using Google Trends search volume for specific terms (e.g. 'debt') to predict market movements.

In [ ]:
from IPython.core.display import HTML
HTML("<iframe src=http://www.nature.com/srep/2013/130425/srep01684/full/srep01684.html width=900 height=400></iframe>")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('figure', figsize=(8, 7))

Tobias Preis was kind enough to provide me with the data used in this publication. There are many Google Trends search words but here we will only be using 'debt'.

In [ ]:
data = pd.read_csv('GoogleTrendsData.csv', index_col='Date', parse_dates=True)
data.head()

djia is the Dow Jones Industrial Average

In [ ]:
data.plot(subplots=True)

The authors detect if search volume is relatively increasing or decreasing in any given week by forming a moving average and testing if the current value crosses the moving average of the past 3 weeks. 

Lets first compute the moving average.

In [ ]:
data['debt_mavg'] = data.debt.rolling(window=3).mean()
data.head()

Since we want to see if the current value is above the moving average of the *preceeding* weeks, we have to shift the moving average timeseries forward by one.

In [ ]:
data['debt_mavg'] = data.debt_mavg.shift(1)
data.head(10)

### Generate the order signals.

From the paper:

"We use Google Trends to determine how many searches n(t – 1) have been carried out for a specific search term such as debt in week t – 1, where Google defines weeks as ending on a Sunday, relative to the total number of searches carried out on Google during that time."

and

"We implement this strategy by selling the DJIA at the closing price p(t) on the first trading day of week t, if Δn(t − 1, Δt) > 0, and buying the DJIA at price p(t + 1) at the end of the first trading day of the following week. [...]. If instead Δn(t − 1, Δt) < 0, then we buy the DJIA at the closing price p(t) on the first trading day of week t and sell the DJIA at price p(t + 1) at the end of the first trading day of the coming week."

In [ ]:
data.loc[:, 'order'] = 0
data.loc[data.debt > data.debt_mavg, 'order'] = -1 # Short if search volume goes up relative to mavg.
data.loc[data.debt < data.debt_mavg, 'order'] = 1 # Long if search volume goes down relative to mavg.
data.head(10)

### Compute returns

In [ ]:
data['ret_djia'] = data.djia.pct_change()
data.head()

Returns at week `t` are relative to week `t-1`. However, we are buying at week `t` and selling at week `t+1`, so we have to adjust by shifting the returns upward.

In [ ]:
data['ret_djia'] = data['ret_djia'].shift(-1)
data.head()

The algorithm that is used by the authors makes a decision every Monday of whether to long or short the Dow Jones. After this week passed, we exit all positions (sell if we longed, buy if we shorted) and make a new trading decision.

The `ret` column contains the weekly returns. Thus, if we buy at week `t` sell at week `t+1` we make the returns of week `t+1`. Conversely, if we short at week `t` and buy back at week `t+1` we make the negative returns of week `t+1`.

In [ ]:
# Compute returns of our strategy
data['ret_google'] = data.order * data.ret_djia
data.head(10)

Now we just have to compound the returns. As we are reinvesting our earns, returns actually do not compound by summing them up but by taking their cumulative product:

$$i_t = (i_{t-1} + i_{t-1} \cdot r_t) = (1 + r_t) \cdot i_{t-1}, \quad i_0 = 1$$

In [ ]:
figure(figsize=(9, 2))
(1 + data.ret_google).cumprod().plot()
plt.ylabel('Portfolio value')

In [ ]:
from IPython.core.display import Image 
Image("http://www.nature.com/srep/2013/130425/srep01684/carousel/srep01684-f2.jpg")

# Credits

This notebook was partly extracted from the Tutorial of Wes McKinney, author of Pandas, at Pycon 2013 in Santa Clara.


The Google Trends example was extracted from the tutorial **Financial Analysis in Python**, given by Thomas Wiecki at PyData Boston 2013